## An Electrode Design Optimisation Example

A design optimisation example loosely based on work by L.D. Couto available at [[1]](https://doi.org/10.1016/j.energy.2022.125966).

The target is to maximise the gravimetric energy density over a range of possible design parameter values, including for example:

cross-sectional area = height x width (only need change one), electrode widths, particle radii, volume fractions and separator width.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [2]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, we import the added packages plus any additional dependencies,

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo

import pybop
import pybamm
from pybamm import Parameter

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [4]:
# Load Chen2020 model
model = pybamm.lithium_ion.DFN()
params = pybamm.ParameterValues("Chen2020")

# Time step (in seconds) for each pulse in the HPPC test
pulse_time = 3500  
initial_soc = 1.0

# Discharge current to simulate 10% discharge
C_rate = 1  # 1C discharge rate (can be adjusted based on your requirements)

experiment = pybamm.Experiment(
    [
    f"Discharge at {C_rate}C for {pulse_time} seconds",
    ],
    period = "1 seconds"
)
sim = pybamm.Simulation(model, parameter_values=params, experiment=experiment)
sim.solve(initial_soc = initial_soc)
solution = sim.solution

# Extract voltage, current, time from the solution
reference_voltage = sim.solution["Terminal voltage [V]"].entries
reference_current = sim.solution["Current [A]"].entries
reference_time = sim.solution["Time [s]"].entries

## Optimising the Parameters

First, we define the model to be used for the parameter optimisation,

In [5]:
# Load Chen2020 model
parameter_set = pybop.ParameterSet.pybamm("Chen2020")

# Change the diffusivities to some random values
parameter_set['Negative particle diffusivity [m2.s-1]'] = 1.68e-12   # actual value - 3.3e-14 (12-16)
parameter_set['Positive particle diffusivity [m2.s-1]'] = 1.32e-16   # actual value - 4e-15 (13-17)

model = pybop.lithium_ion.SPMe(parameter_set=parameter_set)




Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a prior for the parameters. The initial parameters values used in the optimisation will be randomly drawn from the prior distribution.

In [6]:
parameters = pybop.Parameters(
    pybop.Parameter(
        "Negative particle diffusivity [m2.s-1]",
        prior=pybop.Gaussian(1e-14, 3e-13),
        bounds=[1e-16, 1e-12],
    ),
    pybop.Parameter(
        "Positive particle diffusivity [m2.s-1]",
        prior=pybop.Gaussian(1e-15, 3e-14),
        bounds=[1e-17, 1e-13],
    ),
)

dataset = pybop.Dataset(
    {
        "Time [s]": reference_time,
        "Current function [A]": reference_current,
        "Voltage [V]": reference_voltage,
    }
)

Next, we construct the experiment for design optimisation and the initial state-of-charge,

We can now define the output signal, the problem (which combines the model with the dataset) and construct a cost function which in this example is the `GravimetricEnergyDensity()` used to maximise the gravimetric energy density of the cell.

In [7]:
model.build(
    initial_state={"Initial SoC": initial_soc}
)
problem = pybop.FittingProblem(
    model,
    parameters,
    dataset,
)

cost = pybop.SumSquaredError(problem)

Let's construct PyBOP's optimisation class. This class provides the methods needed to fit the forward model. For this example, we use particle swarm optimisation (PSO). Due to the computational requirements of the design optimisation methods, we limit the number of iterations to 15 for this example.

In [8]:
optim = pybop.PSO(cost, verbose=True, max_iterations=200)

Finally, we run the optimisation and return the values obtained,

In [9]:
results = optim.run()
print(results)

Halt: No significant change for 15 iterations.
OptimisationResult:
  Initial parameters: [1.9999e-16 1.9999e-17]
  Optimised parameters: [3.75168506e-14 4.00222400e-15]
  Final cost: 0.1573744719309394
  Optimisation time: 46.24281978607178 seconds
  Number of iterations: 44
  SciPy result available: No
OptimisationResult:
  Initial parameters: [1.9999e-16 1.9999e-17]
  Optimised parameters: [3.75168506e-14 4.00222400e-15]
  Final cost: 0.1573744719309394
  Optimisation time: 46.24281978607178 seconds
  Number of iterations: 44
  SciPy result available: No


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Comparing System Response

We can quickly plot the system's response using the estimated parameters compared to the initial parameters:


In [10]:
pybop.plot.quick(problem, problem_inputs=results.x, title="Optimised Comparison");

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:


In [14]:
bounds = np.asarray([[1e-17, 1e-12], [1e-17, 1e-12]])
pybop.plot.surface(optim,bounds=bounds);